<head><b><center>MACHINE LEARNING</center>

<center>Redes Neuronales Artificiales</center>
    

<center>Profesor: Gabriel Jara </center></b></head><br>
El presente Jupyter Notebook busca:
<ul>
    <li>Retomar conceptos sobre Redes Neuronales Artificiales (ANN).</li>
    <li>Profundizar en el mecanismo de aprendizaje de las ANN</li>
    <li>Analizar la implementación de un Perceptron, unidad básica de las redes neuronales.</li>
</ul>


Algunas imágenes han sido robadas de internet y son enlaces a su correspondiente fuente... por ejemplo esta: 
<br> <img src=https://pbs.twimg.com/media/F-Q_3VXXkAAsRNX.jpg:large
          width="300"/></br>




<b>PERCEPTRON</b>

Cuando en Ciencia de Datos presentamos las Redes Neuronales Artificiales como modelo de aprendizaje, identificamos el perceptrón como unidad básica de este modelo, cuya función emula el rol de una neurona: percibe un estímulo según el cual se activa (o no) y con ello transmite una señal. 
<br> <img src=https://images.deepai.org/glossary-terms/perceptron-6168423.jpg
          width="400"/></br>
          
En Ciencia de Datos construimos modelos usando implementaciones de redes neuronales ya disponibles, como <i>Keras - TensorFlow</i>, y en general es lo que haríamos en un contexto productivo. Pero en esta asignatura queremos profundizar en la topología misma de la red, desentrañar el cómo y porqué aprende, y eso no lo lograríamos sólo importando librerías. Así que hoy comenzaremos a construir nuestra propia implementación de red neuronal, y partiremos con una simple neurona. 

Definamos una clase <i>Perceptron</i>, que se inicialice con parámetros que definen tasa de aprendizaje y cantidad de épocas de entrenamiento por defecto (más adelante volveremos sobre esto), así como pesos que se asocian a cada entrada y un peso adicional de sesgo. Estos pesos se multiplican por cada valor de entrada y se suman con el sesgo, para luego ser aplicada la función de activación, en este caso una función de paso, que determina si el perceptrón se activa o no a partir de los valores de entrada.    



In [2]:
import numpy as np

# Definimos la clase Perceptron
class Perceptron:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        # Valores por defecto en caso que no se especifique Learnng rate o n_iters
        self.learning_rate = learning_rate
        self.n_iters = n_iters
        # Pesos y sesgo
        self.weights = None
        self.bias = None
            
    def _unit_step_function(self, x):
        # Esta es la función de activación
        return np.where(x >= 0, 1, 0)

    def fit(self, X, y):
        # Método que se empleará para el entrenamiento (ajuste) del Perceptrón
        # Se identifica cuantos datos hay en la muestra y con cuantos atributos
        n_samples, n_features = X.shape
        
        # Se inicializan tantos pesos como atributos...
        self.weights = np.zeros(n_features)
        # ...más un sesgo
        self.bias = 0
        # Por ahora hemos iniciado todos los pesos y sesgo en valor cero. 
        # Más adelante debieramos revisar esta decisión. 
        
        for _ in range(self.n_iters):
            # Iteraremos haciendo ajuste de los pesos
            for idx, x_i in enumerate(X):
                # Primero calculamos la salida de la transformación lineal: X*W + B
                linear_output = np.dot(x_i, self.weights) + self.bias
                # La salida de esa transformación lineal la pasamos por la función de activación.
                y_predicted = self._unit_step_function(linear_output)
                # Calculamos la magnitud de actualización, que es proporcional al learning rate. 
                # Note que Si y == y_hat; => update == 0. 
                delta = self.learning_rate * (y[idx] - y_predicted)
                # Actualizamos pesos y sesgo. 
                self.weights += delta * x_i
                self.bias += delta

    def predict(self, X):
        # Método que usaremos para obtener predicciones con el Perceptrón
        # Primero calculamos la salida de la transformación lineal: X*W + B
        linear_output = np.dot(X, self.weights) + self.bias
        # La salida de esa transformación lineal la pasamos por la función de activación.
        y_predicted = self._unit_step_function(linear_output)
        # y esa es nuestra predicción, así que la retornamos. 
        return y_predicted

ModuleNotFoundError: No module named 'numpy'

El perceptrón debe tener una función de activación, que corresponde a la transformación no lineal que se aplica sobre la transformación lineal determinada por pesos y sesgo, y que típicamente es una función de paso binaria, es decir que devuelve cero o uno según si su entrada supera o no un umbral. 

El perceptrón tiene dos métodos adicionales. Uno de ellos es para el entrenamiento en que ajusta sus pesos para resolver un problema a partir de la muestra $X$ e $y$ que reciba. El otro método es para realizar predicciones basada en una muestra $X$.

<b>¿En qué consiste predecir con el perceptrón?</b>

Tenemos datos en una matriz $X$ donde cada columna son atributos, cada fila son observaciones. El perceptrón ejecuta una transformación lineal de cada observación en $X$ multiplicando cada atributo por un coeficiente de peso y sumando un sesgo. El resultado de la transformación lineal alimenta a una función de paso que lo transforma en cero o uno, y esta salida es la predicción del valor que tendrían las observaciones en un hipotético vector $y$. 

<b>¿Cómo se ajusta los pesos del perceptrón?</b>

Tenemos datos en una matriz $X$ donde cada columna son atributos, cada fila son observaciones y cada una de estas observaciones se corresponde con un valor en el vector $y$ (semántica posicional). El perceptrón ajusta los pesos buscando que dado $X$ como entrada a la salida se obtenga $y$. Para esto usa la <b>Regla Delta</b>.

La regla delta es una forma ingeniosa de decretar la dirección y magnitud del ajuste requerido por los pesos. Suponga que dado cierta entrada $X$ observamos que a la salida del perceptrón se obtiene un valor 0, pero el valor $y$ esperado era 1, entonces diríamos que se cometió un error de 1. Lo que quisíeramos es que los parámetros de peso se ajusten al alza, para aumentar la probabilidad de que ante la misma entrada $X$ la respuesta del perceptrón sea 1. Si por otro lado el perceptrón responde 1 cuando debería haber sido 0, entonces diríamos que se cometió un error de magnitud -1 y quisíeramos que los pesos se ajusten a la baja para aumentar la probabilidad de que responda 0 para esa entrada. 

El error lo estamos definiendo como la diferencia entre el valor real $y$ y la predicción que genera el perceptrón. 

$$ error = y - \hat{y}$$

Entonces lo que hacemos es ajustar los pesos de la red en la dirección del error. La magnitud del ajuste queda controlada por un parámetro $\alpha$ que conocemos como <i>Learning Rate</i>. Entonces, cada peso $i$ se actualiza de acuerdo con la fórmula:

$$w_{i} \leftarrow w_{i} - \alpha (y - \hat{y}) $$

Cuando el perceptrón acierta en el pronóstico significa que $y = \hat{y}$, y por lo mismo en ese caso la actualización de pesos se vuelve nula. Así, los pesos del perceptrón se ajustan cada vez que éste se equivoca, en la dirección adecuada para reducir el error, y no se ajustan cuando el perceptrón acierta. 

<br> <img src=https://media.springernature.com/lw685/springer-static/image/art%3A10.1007%2Fs00521-022-07233-1/MediaObjects/521_2022_7233_Fig2_HTML.png
          width="300"/></br>

En nuestra próxima clase veremos como la idea de la regla delta se extiende usando el concepto de función de error y gradiente, habilitando así el aprendizaje a nivel de redes de perceptrones. Pero por ahora, sigamos analizando el caso de un perceptrón. 

<b>¿Para qué podemos usar este simple perceptrón? </b>

Por ejemplo, para resolver algunas operaciones lógicas como la operación <i>AND</i>. A continuación tenemos operación lógica <i>AND</i> muy simple, con sólo cuatro posibles entradas y dos posibles salidas, que usamos para entrenar al perceptrón y luego observar que replica el resultado perfectamente. 

In [ ]:
# Datos de entrenamiento (X: atributos, y: etiquetas)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 0, 0, 1])  # AND lógico

# Creamos una instancia del perceptrón y lo entrenamos
perceptron = Perceptron(learning_rate=0.1, n_iters=10)


perceptron.fit(X, y)

# Realizamos predicciones
predicciones = perceptron.predict(X)
print("Predicciones de AND:", predicciones)

En lugar de <i>AND</i> podríamos aprender <i>OR</i>. 

In [ ]:
# Datos de entrenamiento (X: atributos, y: etiquetas)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 1])  # OR lógico

# Creamos una instancia del perceptrón y lo entrenamos
perceptron = Perceptron(learning_rate=0.1, n_iters=10)
perceptron.fit(X, y)

# Realizamos predicciones
predicciones = perceptron.predict(X)
print("Predicciones de OR:", predicciones)

Se aprecia que nuestro perceptrón no tiene dificultades en aprender estas dos operaciones lógicas. 

¿Cómo generalizamos un poco este experimento? Definamos una operación de tipo <i>AND</i> algo más compleja, que dé verdadero si $x$ e $y$ son positivo, en cualquier otro caso sea falso. Sería una versión continua de la misma función <i>AND</i>. Podemos visualizar datos de ejemplo de dicha función <i>AND</i> continua en la gráfica a continuación. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generamos 100 puntos aleatorios en el plano XY
X = np.random.uniform(-1, 1, (100, 2))

# Calculamos los valores de la función AND
y = np.logical_and(X[:, 0] > 0, X[:, 1] > 0)

# Configuramos el gráfico
plt.figure(figsize=(8, 8))

# Recorremos los puntos y los dibujamos con su respectivo símbolo y color
for i in range(X.shape[0]):
    if y[i]:
        plt.text(X[i, 0], X[i, 1], 'O', color='blue', fontsize=12, ha='center', va='center')
    else:
        plt.text(X[i, 0], X[i, 1], 'X', color='red', fontsize=12, ha='center', va='center')

# Configuramos los límites y las líneas del eje
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plt.grid(color='gray', linestyle='--', linewidth=0.5)

# Añadimos la leyenda
plt.text(1, 1.2, 'O: Verdadero', color='blue', fontsize=12, ha='left')
plt.text(1, 1.1, 'X: Falso', color='red', fontsize=12, ha='left')

# Añadimos título y etiquetas
plt.title('Representación de la función AND en el plano XY')
plt.xlabel('X')
plt.ylabel('Y')

# Mostramos el gráfico
plt.show()


¿Puede nuestro perceptrón aprender esta nueva función <i>AND</i>? Nada nos impide intentarlo. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Entrenamos un Perceptrón que aprenda AND
perceptron = Perceptron(learning_rate=0.1, n_iters=10)
perceptron.fit(X, y)

# Utilizamos el Perceptron ya entrenado
y_hat = perceptron.predict(X)

# Configuramos el gráfico
plt.figure(figsize=(8, 8))

# Recorremos los puntos y los dibujamos con su respectivo símbolo y color
for i in range(X.shape[0]):
    if y_hat[i]:
        plt.text(X[i, 0], X[i, 1], 'O', color='blue', fontsize=12, ha='center', va='center')
    else:
        plt.text(X[i, 0], X[i, 1], 'X', color='red', fontsize=12, ha='center', va='center')

# Configuramos los límites y las líneas del eje
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plt.grid(color='gray', linestyle='--', linewidth=0.5)

# Añadimos la leyenda
plt.text(1, 1.2, 'O: Verdadero', color='blue', fontsize=12, ha='left')
plt.text(1, 1.1, 'X: Falso', color='red', fontsize=12, ha='left')

# Añadimos título y etiquetas
plt.title('Función AND en el plano XY según nuestro Perceptron')
plt.xlabel('X')
plt.ylabel('Y')

# Mostramos el gráfico
plt.show()

Se nota que sí está tratando de aprender la función <i>AND</i> pero dejó mal clasificados varios puntos. Podríamos especular que es porque no ha entrenado lo suficiente, dado que le redujimos la cantidad de épocas a sólo 10, ¿qué pasa si las aumentamos?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Entrenamos un Perceptrón que aprenda AND
perceptron = Perceptron(learning_rate=0.1, n_iters=1000)
perceptron.fit(X, y)

# Utilizamos el Perceptron entrenado
y_hat = perceptron.predict(X)

# Configuramos el gráfico
plt.figure(figsize=(8, 8))

# Recorremos los puntos y los dibujamos con su respectivo símbolo y color
for i in range(X.shape[0]):
    if y_hat[i]:
        plt.text(X[i, 0], X[i, 1], 'O', color='blue', fontsize=12, ha='center', va='center')
    else:
        plt.text(X[i, 0], X[i, 1], 'X', color='red', fontsize=12, ha='center', va='center')

# Configuramos los límites y las líneas del eje
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plt.grid(color='gray', linestyle='--', linewidth=0.5)

# Añadimos la leyenda
plt.text(1, 1.2, 'O: Verdadero', color='blue', fontsize=12, ha='left')
plt.text(1, 1.1, 'X: Falso', color='red', fontsize=12, ha='left')

# Añadimos título y etiquetas
plt.title('Función AND en el plano XY según nuestro Perceptron')
plt.xlabel('X')
plt.ylabel('Y')

# Mostramos el gráfico
plt.show()

Salvo que se demoró más, el resultado es esencialmente el mismo, con mínimas diferencias (si es que las hubiere). No parece ser un problema de cuanto entrenemos, y lo podemos apreciar si consideramos cómo trabaja el perceptrón, que es en esencia lineal. 

Observemos que los valores asociados a los pesos (incluido el sesgo), relacionan los valores $X$ con la salida $y$ de la red usando una línea recta. Podemos apreciar dicha recta directamente. 

In [ ]:
# Configuramos el gráfico
plt.figure(figsize=(8, 8))

# Recorremos los puntos y los dibujamos con su respectivo símbolo y color
for i in range(X.shape[0]):
    if y_hat[i]:
        plt.text(X[i, 0], X[i, 1], 'O', color='blue', fontsize=12, ha='center', va='center')
    else:
        plt.text(X[i, 0], X[i, 1], 'X', color='red', fontsize=12, ha='center', va='center')
        
# Configuramos los límites y las líneas del eje
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plt.grid(color='gray', linestyle='--', linewidth=0.5)

# Añadimos la leyenda
plt.text(1, 1.3, 'O: Verdadero', color='blue', fontsize=12, ha='left')
plt.text(1, 1.2, 'X: Falso', color='red', fontsize=12, ha='left')
plt.text(1, 1.1, '--'': Frontera de decisión', color='black', fontsize=12, ha='left')

# Dibujamos la frontera de decisión
# No confunda XY del plano con X e y del problema. Para evitar confuciones 
# llamaremos al plano x0 y x1. 
x0 = np.array([-1, 1])
# para los valores x0 = -1 y 1, calcularemos x1 usando 
# la ecuación de la recta y = w0*x0 + w1*x1 + b, con y = 0. 
# Note que es la misma recta que usa el perceptrón cuando hace la predicción. 
x1 = -(perceptron.weights[0] * x0 + perceptron.bias) / perceptron.weights[1]
# Añadimos esta recta al gráfico, para visualizar así la frontera de decisión del perceptrón.
plt.plot(x0, x1, color='black', linestyle='--')

# Añadimos título y etiquetas
plt.title('Función AND en el plano XY según nuestro Perceptron')
plt.xlabel('X')
plt.ylabel('Y')

# Mostramos el gráfico
plt.show()

Ahora podemos ver con más claridad porque el perceptrón no llega a resolver por completo nuestra versión continua de <i>AND</i>, lo que pasa es que no hay forma de trazar una línea recta que separe perfectamente los verdadero de los falso. No importa cuanto se entrene, el perceptrón no va a poder aprender a la perfección esta función, por no ser ella linealmente separable. 

Por otro lado, veamos el vaso medio lleno, el perceptrón sí ha aprendido, puesto que identifica la frontera que minimiza el error, es decir logra clasificar correctamente la mayoría de los puntos posible con una línea recta. Puede parecer poca cosa, pero este es el principio de la inteligencia, al menos la artificial (probablemente también la biológica). 

Así que diremos que el perceptrón sí resuelve <i>AND</i>, ya sabemos que también resuelve <i>OR</i>. ¿Qué operación lógica no resuelve?

Se conoce como <i>XOR</i> al "O estricto", es decir la relación "o" y sólo "o", excluyendo el caso en que "y" da verdadero. Esta operación lógica no es linealmente separable en su versión discreta, y por lo mismo no ofrece al perceptrón ninguna posibilidad de aprendizaje.   

In [ ]:
# Datos de entrenamiento (X: atributos, y: etiquetas)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0])  # XOR lógico

# Creamos una instancia del perceptrón y lo entrenamos
perceptron = Perceptron(learning_rate=0.1, n_iters=10)
perceptron.fit(X, y)

# Realizamos predicciones
predicciones = perceptron.predict(X)
print("Predicciones de XOR:", predicciones)

Así como no funciona en la versión discreta de <i>XOR</i>, tampoco lo hará para nuestra adaptación continua. El resultado que obtengamos será simplemente aleatorio, puesto que el perceptrón no tiene forma de aprender esta función, dado que No existe la línea recta que minimiza el error de predicción para este caso. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generamos 100 puntos aleatorios en el plano XY
X = np.random.uniform(-1, 1, (100, 2))

# Calculamos los valores de la función AND
y = np.logical_xor(X[:, 0] > 0, X[:, 1] > 0)

# Configuramos el gráfico
plt.figure(figsize=(8, 8))

# Recorremos los puntos y los dibujamos con su respectivo símbolo y color
for i in range(X.shape[0]):
    if y[i]:
        plt.text(X[i, 0], X[i, 1], 'O', color='blue', fontsize=12, ha='center', va='center')
    else:
        plt.text(X[i, 0], X[i, 1], 'X', color='red', fontsize=12, ha='center', va='center')

# Configuramos los límites y las líneas del eje
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plt.grid(color='gray', linestyle='--', linewidth=0.5)

# Añadimos la leyenda
plt.text(1, 1.2, 'O: Verdadero', color='blue', fontsize=12, ha='left')
plt.text(1, 1.1, 'X: Falso', color='red', fontsize=12, ha='left')

# Añadimos título y etiquetas
plt.title('Representación de la función XOR en el plano XY')
plt.xlabel('X')
plt.ylabel('Y')

# Mostramos el gráfico
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Entrenamos un Perceptrón que aprenda XOR
perceptron = Perceptron(learning_rate=0.1, n_iters=10)
perceptron.fit(X, y)

# Utilizamos el Perceptron que entrenado
y_hat = perceptron.predict(X)

# Configuramos el gráfico
plt.figure(figsize=(8, 8))

# Recorremos los puntos y los dibujamos con su respectivo símbolo y color
for i in range(X.shape[0]):
    if y_hat[i]:
        plt.text(X[i, 0], X[i, 1], 'O', color='blue', fontsize=12, ha='center', va='center')
    else:
        plt.text(X[i, 0], X[i, 1], 'X', color='red', fontsize=12, ha='center', va='center')

# Configuramos los límites y las líneas del eje
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plt.grid(color='gray', linestyle='--', linewidth=0.5)

# Añadimos la leyenda
plt.text(1, 1.1, 'O: Verdadero', color='blue', fontsize=12, ha='left')
plt.text(1, 1.2, 'X: Falso', color='red', fontsize=12, ha='left')


# Añadimos título y etiquetas
plt.title('Función XOR en el plano XY según nuestro Perceptron')
plt.xlabel('X')
plt.ylabel('Y')

# Mostramos el gráfico
plt.show()

Así que podemos concluir que el perceptrón es bastante limitado, sólo puede trazar líneas rectas y sólo logra aprender a resolver problemas que sean linealmente separables. 

<b>PERCEPTRÓN PARA CLASIFICACIÓN</b>

El modelo que estamos estudiando es adecuado en problemas de clasificación. <i>¿Recuerda algún modelo de clasificación, visto en Ciencia de Datos, que se pareciera al Perceptrón?</i>

Veamos como le va a nuestra implementación de perceptron en comparación. 


<b>Datos para probar</b>

Cuando parecía que ya se había terminado de hundir, vamos a reflotar al Titanic, una vez más. Este set de datos ya lo conocemos, puesto que lo usamos muchas veces como ejemplo en la asignatura Ciencia de Datos. 

In [ ]:
import numpy as np  
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

# Una vez más cargamos y procesamos el Titanic
file = open("Titanic.csv", "r")
titanic = file.read()
file.close()
titanic = titanic.split("\n")
for j in range(len(titanic)):
    titanic[j] = titanic[j].split(",")

# El nombre tiene ',' así que necesitamos volver a unirlo.
for elemento in titanic[1:]:
    elemento[3] = elemento[3]+elemento[4]
    del elemento[4]
titanic = np.array(titanic)
titanic = np.delete(titanic, [3,8,10,11], axis=1)

# Codificamos género como 0-1
for linea in titanic[1:]:
    linea[3] = '0' if linea[3] == 'male' else '1'

# Quitamos las observaciones con datos perdidos
borrar = []
for l in range(len(titanic)):
    if '' in titanic[l]:
        borrar.append(l)
titanic = np.delete(titanic, borrar, axis=0)

# Arreglamos vector y matriz como array
y = titanic[:,1][1:]
X = titanic[:,2:][1:]
X, y = np.array(X, dtype='float64'), np.array(y, dtype='float64')

# Aplicamos Normalización Min-Max
X= MinMaxScaler().fit_transform(X)

# Separamos la muestra al azar, 60% para entrenar, 40% para testeo final.
# La proporción para entrenamiento es alta dado que el Perceptrón es sensible
# a datos "adversos", así que estamos tratando de reducir ese riesgo.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)


<b>Modelo Base para comparar</b>

Cualquier resultado que obtengamos con el Perceptrón, ¿cómo podríamos calificarlo de bueno o malo? En general no se puede, pero sí podemos decir observar si se comporta mejor, peor o similar a otros modelos. Por eso es bueno tener un modelo que sirvan de línea base de comparación. En problemas de clasificación se suele usar Regresión Logística para este fin. 

El Perceptrón, como hemos visto, es un modelo de clasificación líneal, es decir se basa en una frontera de decisión caracterizada como una recta. Tiene por tanto doble sentido el comparar el perceptrón con la regresión logística, dado que ambos son modelos lineales.  

In [ ]:
from sklearn.linear_model import LogisticRegression

#Se define y entrena la Regresión Logística
lg_model = LogisticRegression().fit(X_train, y_train)

# Utilizamos el modelo para predecir
y_hat = lg_model.predict(X_test)


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_test, y_hat)

print('MATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de testeo con Regresión Logística es: {:.3f}'.format(accuracy_score(y_test,y_hat)))

Con regresión logística logramos clasificar con una exactitud mayor de lo que se podría justificar con simple azar. La teoría es que el perceptrón debiera lograr resultados similares. Nótese que la exactitud reportada corresponde a la que se obtiene con datos de testeo, distintos a los que se usó para ajustar la regresión.

Debemos tener en cuenta, eso sí, que el perceptrón y especialmente nuestra rudimentaria implemetación, podría enfrentar problemas de optimización que le impidan converger a una solución adecuada. El experimento que se ha organizado con una distribución de 60-40% entre entrenamiento y testeo, en parte para tratar de evitar distribuciones adversas en los datos. Si ocurriera que nuestro perceptrón no aprende (clasifica mal), repitiendo el experimento desde la organización de los set de datos podríamos obtener otra versión que sí aprenda. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Entrenamos un Perceptrón con los datos de Titanic
perceptron = Perceptron(learning_rate=0.01, n_iters=1000)
perceptron.fit(X_train, y_train)

# Utilizamos el Perceptron para predecir
y_hat = perceptron.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_test, y_hat)

print('MATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de testeo con Perceptrón es: {:.3f}'.format(accuracy_score(y_test,y_hat)))

Si hemos repetido el experimento con el perceptrón un par de veces, es altamente probable que observemos resultados equivalentes a los que produce regresión logística. Con esto estamos validando que el perceptrón tiene la capacidad de resolver problemas linealmente separables, al igual que lo hace regresión logística. 

Por ahora no tenemos una implementación que asegure la convergencia del aprendizaje, y es que nuestro enfoque para la optimización de pesos del perceptrón es todavía muy básico, tiene el potencial de funcionar pero no garantía de que ocurra. Esto se refleja en que cuando repetimos el experimento nuestro perceptrón no siempre logra el aprendizaje. 

<b>PERCEPTRON PARA REGRESIÓN</b>

Ya hemos visto que el perceptrón puede resolver problemas de clasificación, <i>¿puede el perceptrón resolver problemas de regresión?, ¿qué modificación habría que realizar para lograrlo?</i>

Vamos a utilizar los datos de producción de energía en una planta de ciclo combinado, que ya habíamos usado para regresión en Ciencia de Datos.  

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Importar datos desde csv
file = open("Ciclo_Combinado.csv", "r")
obs = file.read()
file.close()
obs = obs.split("\n")
# hay un salto de línea demás en el archivo
obs.remove(obs[-1])
# dividir los datos por coma. 
for j in range(len(obs)):
    obs[j] = obs[j].split(",")
# transformar a numpy array.
obs = np.array(obs)
# Valores Y (variable dependiente)
y = obs[:,-1][1:]
# Matriz X (variables independientes)
X = obs[:,:-1][1:]

# Los valores de X e y tienen que ser numéricos
X, y = np.array(X, dtype='float64'), np.array(y, dtype='float64')

# Usamos Min-Max para estandarizar el rango de datos 
X= MinMaxScaler().fit_transform(X)

# Usaremos 80% para entrenar y 20% para testear
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

ModuleNotFoundError: No module named 'numpy'

Sabemos que este tipo de problema se puede resolver usando Regresión Lineal, modelo que usaremos como base para comparar los resultados que ustedes obtendrán adaptando el Perceptrón. 

In [ ]:
from sklearn.linear_model import LinearRegression

#Se define el modelo
model = LinearRegression().fit(X_train, y_train)
r_sq = model.score(X_test, y_test)
print('Coeficiente de determinación (R2):', r_sq)
print('Intercepto (Beta0):', model.intercept_)
print('Pendiente (Betas 1 en adelante):', model.coef_)

<b>Actividad de aprendizaje práctico</b>

Modifique la implementación de nuestro perceptrón para que se pueda utilizar como modelo de regresión. Compare los resultados con los que se obtiene regresión lineal.  

Para mayor provecho de esta tarea evitaremos buscar la solución ya implementada o descansar en servicios AI. Discutamos qué necesita ser modificado y demostremos que podemos hacer ese cambio por nuestra cuenta. 

In [ ]:
# Haga acá su experimento con perceptrón para regresión,
# agregue celdas que necesite,
# incluya comparación con regresión lineal. 

# Modifique la clase Perceptrón para usar en regresión
class Perceptron:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        # Valores por defecto en caso que no se especifique Learnng rate o n_iters
        self.learning_rate = learning_rate
        self.n_iters = n_iters
        # Pesos y sesgo
        self.weights = None
        self.bias = None
            
    def _unit_step_function(self, x):
        # Esta es la función de activación
        return np.where(x >= 0, 1, 0)

    def fit(self, X, y):
        # Método que se empleará para el entrenamiento (ajuste) del Perceptrón
        # Se identifica cuantos datos hay en la muestra y con cuantos atributos
        n_samples, n_features = X.shape
        
        # Se inicializan tantos pesos como atributos...
        self.weights = np.zeros(n_features)
        # ...más un sesgo
        self.bias = 0
        # Por ahora hemos iniciado todos los pesos y sesgo en valor cero. 
        # Más adelante debieramos revisar esta decisión. 
        
        for _ in range(self.n_iters):
            # Iteraremos haciendo ajuste de los pesos
            for idx, x_i in enumerate(X):
                # Primero calculamos la salida de la transformación lineal: X*W + B
                linear_output = np.dot(x_i, self.weights) + self.bias
                # La salida de esa transformación lineal la pasamos por la función de activación.
                y_predicted = self._unit_step_function(linear_output)
                # Calculamos la magnitud de actualización, que es proporcional al learning rate. 
                # Note que Si y == y_hat; => update == 0. 
                delta = self.learning_rate * (y[idx] - y_predicted)
                # Actualizamos pesos y sesgo. 
                self.weights += delta * x_i
                self.bias += delta

    def predict(self, X):
        # Método que usaremos para obtener predicciones con el Perceptrón
        # Primero calculamos la salida de la transformación lineal: X*W + B
        linear_output = np.dot(X, self.weights) + self.bias
        # La salida de esa transformación lineal la pasamos por la función de activación.
        y_predicted = self._unit_step_function(linear_output)
        # y esa es nuestra predicción, así que la retornamos. 
        return y_predicted

<b>CONCLUSIÓN</b>

Hemos presentado el Perceptrón, modelo matemático que es la unidad básica de las redes neuronales artificales. Hemos visto que el perceptrón puede realizar tareas de clasificación y regresión, pero también observamos que su limitación natural sólo permite que realice estas tareas en forma lineal. Es decir, sólo puede separar aquello que es linealmente separable.

Hemos visto que el perceptrón aprende a partir de observar datos previamente clasificados (aprendizaje supervisado), y lo hace ajustando sus parámetros internos (pesos y sesgo) iterativamente, usando el error de salida (diferencia entre predicción y realidad) para indicar la dirección adecuada en ese ajuste. Tenemos a la vista el código (Python) de toda esta operación, por lo que no hay misterio respecto al cómo aprende el perceptrón. 

Podemos modificar nuestra implementación de perceptrón para que satisfaga diversos propósitos, usarlo para regresión en lugar de clasificación es sólo un ejemplo de lo que podemos hacer. 

En nuestra próxima clase extenderemos el concepto de perceptrón a capas, permitendo así conformar redes neuronales con múltiples capas. Por ahora seguiremos trabajando en nuestra propia implementación, para así asegurarnos de entender cómo ocurre el aprendizaje en nuestro modelo de red neuronal. Cuando más adelante aprovechemos modelos más avanzados y que ya están implementados, siempre sobre la base de conocer suficientemente su funcionamiento para asegurar que no nos parezca magia. 
